In [1]:
import torch
import torch.nn as nn
import pandas as pd

In [2]:
df = pd.read_csv('fen_analysis.csv')
df

,fen_value,score
0,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...,-33
1,rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBN...,34
2,rnbqkbnr/pppp1ppp/8/4p3/4P3/5N2/PPPP1PPP/RNBQK...,-30
3,r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNB...,30
4,r1bqkbnr/pppp1ppp/2n5/4p3/2B1P3/5N2/PPPP1PPP/R...,-29
...,...,...
336898,8/8/8/8/3pk3/8/3K4/8 b - - 3 60,2
336899,8/8/8/8/3p4/5k2/3K4/8 w - - 4 61,1
336900,8/8/8/8/3p4/3K1k2/8/8 b - - 5 61,-2
336901,8/8/8/8/3p1k2/3K4/8/8 w - - 6 62,0


In [ ]:
df['fen_value'][0]

'rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1'

In [ ]:
def encode(fen):
    